# Many Endings with different Image creation values

## Step 1.1
### Load nessecary modules

In [24]:
# Add python modules
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from copy import deepcopy
# Add parent directory for custom modules
from sys import path as sysPath
sysPath.append('../')
sysPath.append('../Machine_Score/')

# Load custom modules that simr has loaded
import main_SIMR as SIMR
gm = SIMR.gm
im = SIMR.im
SIM = SIMR.SIM
IMG = SIMR.IMG
ms = SIMR.ms
sa = SIMR.sa
dc = SIMR.ms.dc

SIMR.test()
gm.test()
SIM.test()
IMG.test()
ms.test()
dc.test()
sa.test()

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-j0y07sl7 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions
GM: Hi!  You're in Matthew's module for generally useful functions and classes
SIM: Hi!  You're in Matthew's main code for all things simulation.
IMG: Hi!  You're in Matthew's main code for all things image creation.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
DC: Hi!  You're in direct_image_compare.py
SA: Hi!  You're in Matthew's Main program for score analysis!


___
## Step 1.2
### Load Sample Target and Run Info class
Needed for loading the target image and running through models later.

In [25]:
tDir = '../targetDir/'
#tDir = '../dataDir/588013383816904792/'
tDir = gm.validPath(tDir)

tInfo = im.target_info_class( targetDir = tDir, printAll=False)
if tInfo.status == False:
    print("WARNING: target info class bad")
else:
    print("Target Good!: %s" % tInfo.get('target_id'))
    
# Get run info class
rInfo = tInfo.getRunInfo( )

if rInfo.status == False:
    print("WARNING")
else:
    print("Run '%s' Good!"%rInfo.get('run_id'))

Target Good!: 587722984435351614
Run 'run_0000' Good!


___
## Step 2
### Load a starting Image parameter set

In [3]:
# Get memory usage
def getMemUsage():
    import psutil
    mem = psutil.virtual_memory()
    memUsed = mem.used / 1024**3
    memTotal = mem.total / 1024**3
    memPercent = mem.percent
    # return memPercent
    return memUsed, memTotal, memPercent
    # print( 'memUsed: ', memUsed )
    # print( 'memTotal: ', memTotal )
    # print( 'memPercent: ', memPercent )

getMemUsage()

(0.8724365234375, 62.67174530029297, 4.7)

In [4]:
sParam = tInfo.readScoreParam( 'dl-s2-1' )
for key in sParam:
    print( key )

dl-s2-1-base-0
dl-s2-1-base-1
dl-s2-1-base-2
dl-s2-1-base-3
dl-s2-1-base-4
dl-s2-1-base-5
dl-s2-1-base-6
dl-s2-1-base-7
dl-s2-1-base-8
dl-s2-1-base-9
dl-s2-1-base-10
dl-s2-1-base-11
dl-s2-1-base-12
dl-s2-1-base-13
dl-s2-1-base-14
dl-s2-1-base-15
dl-s2-1-base-16
dl-s2-1-base-17
dl-s2-1-base-18
dl-s2-1-base-19
dl-s2-1-base-20
dl-s2-1-base-21
dl-s2-1-base-22
dl-s2-1-base-23
dl-s2-1-base-24
dl-s2-1-base-25
dl-s2-1-base-26
dl-s2-1-many-0
dl-s2-1-many-1
dl-s2-1-many-2
dl-s2-1-many-3
dl-s2-1-many-4
dl-s2-1-many-5
dl-s2-1-many-6
dl-s2-1-many-7
dl-s2-1-many-8
dl-s2-1-many-9
dl-s2-1-many-10
dl-s2-1-many-11
dl-s2-1-many-12
dl-s2-1-many-13
dl-s2-1-many-14
dl-s2-1-many-15
dl-s2-1-many-16
dl-s2-1-many-17
dl-s2-1-many-18
dl-s2-1-many-19
dl-s2-1-many-20
dl-s2-1-many-21
dl-s2-1-many-22
dl-s2-1-many-23
dl-s2-1-many-24
dl-s2-1-many-25
dl-s2-1-many-26


In [5]:
getMemUsage()

(0.8723297119140625, 62.67174530029297, 4.7)

In [4]:
import numpy as np, json
dataLoc = "/home/mbo2d/galStuff/data-dir/"
print("dataLoc: ", dataLoc )

with open( dataLoc + 'dl-s2-2.json' ) as jFile: info = json.load( jFile )
print( 'Info: ', info.keys(), len( info['info'] ) )

imgs = np.load( dataLoc + 'dl-s2-2.npz' )['arr_0']
print( 'Imgs: ', imgs.shape, type( imgs[0,0,0,0] )

dataLoc:  /home/mbo2d/galStuff/data-dir/
Info:  dict_keys(['info']) 52814
Imgs:  (52814, 128, 128, 3)


In [17]:
# Create mask to place images in 10 even containers
m = np.zeros( 52814 ).astype( np.uint8 )
for i in range( 5280 ):
    m[10*i:10*i+10] = np.random.permutation( 10 )

# Ignore last 14 values for symmetry
for i in range( 52800, 52814 ):
    m[i] = 11
    
print( m[0:30] )
print( m[-30:] )
    
    

[7 0 9 8 1 2 5 3 6 4 3 8 0 5 2 4 1 7 6 9 1 2 4 6 5 7 0 9 8 3]
[ 4  2  6  5  3  0  9  7  0  1  8  2  4  5  3  6 11 11 11 11 11 11 11 11
 11 11 11 11 11 11]


In [26]:
print( tInfo.get('target_id') )

587722984435351614


In [28]:
# Init Score List
infoList = []
for i in range( 10 ):
    infoList.append( { 'info' : [] } )

for i in range( 52800 ):
    infoList[ m[i] ]['info'].append( info['info'][i] )
    infoList[ m[i] ]['info'][-1]['target_id'] = tInfo.get('target_id')
    

for i in range( 10 ):
    print( i, len( infoList[i]['info'] ) )

0 5280
1 5280
2 5280
3 5280
4 5280
5 5280
6 5280
7 5280
8 5280
9 5280


In [19]:
imgList = []
for i in range(10):
    imgList.append( imgs[ m == i ] )
    print( imgList[-1].shape )

(5280, 128, 128, 3)
(5280, 128, 128, 3)
(5280, 128, 128, 3)
(5280, 128, 128, 3)
(5280, 128, 128, 3)
(5280, 128, 128, 3)
(5280, 128, 128, 3)
(5280, 128, 128, 3)
(5280, 128, 128, 3)
(5280, 128, 128, 3)


In [29]:
# Save as seperate files

for i in range( 10 ):
    print( i, len( infoList[i]['info'] ), imgList[i].shape, type(imgList[i][0,0,0,0]) )
    imgLoc = dataLoc + 'dl-s2-1%d.npz'  % i
    infLoc = dataLoc + 'dl-s2-1%d.json' % i
    print( '\t - ', imgLoc, infLoc )
    # np.savez_compressed( imgLoc, imgList[i]  )
    with open( infLoc , 'w' ) as jFile: json.dump( infoList[i], jFile )

0 5280 (5280, 128, 128, 3) <class 'numpy.uint8'>
	 -  /home/mbo2d/galStuff/data-dir/dl-s2-10.npz /home/mbo2d/galStuff/data-dir/dl-s2-10.json
1 5280 (5280, 128, 128, 3) <class 'numpy.uint8'>
	 -  /home/mbo2d/galStuff/data-dir/dl-s2-11.npz /home/mbo2d/galStuff/data-dir/dl-s2-11.json
2 5280 (5280, 128, 128, 3) <class 'numpy.uint8'>
	 -  /home/mbo2d/galStuff/data-dir/dl-s2-12.npz /home/mbo2d/galStuff/data-dir/dl-s2-12.json
3 5280 (5280, 128, 128, 3) <class 'numpy.uint8'>
	 -  /home/mbo2d/galStuff/data-dir/dl-s2-13.npz /home/mbo2d/galStuff/data-dir/dl-s2-13.json
4 5280 (5280, 128, 128, 3) <class 'numpy.uint8'>
	 -  /home/mbo2d/galStuff/data-dir/dl-s2-14.npz /home/mbo2d/galStuff/data-dir/dl-s2-14.json
5 5280 (5280, 128, 128, 3) <class 'numpy.uint8'>
	 -  /home/mbo2d/galStuff/data-dir/dl-s2-15.npz /home/mbo2d/galStuff/data-dir/dl-s2-15.json
6 5280 (5280, 128, 128, 3) <class 'numpy.uint8'>
	 -  /home/mbo2d/galStuff/data-dir/dl-s2-16.npz /home/mbo2d/galStuff/data-dir/dl-s2-16.json
7 5280 (5280,

In [31]:
print( infoList[0]['info'][0] )

{'score': 0.9459459459459459, 'model_data': '-9.93853,-4.5805,3.27377,-0.50008,-2.45565,-1.07799,23.33004,24.69427,3.49825,5.32056,309.6923,36.8125,41.78471,51.42857,0.3,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.94594,0,0,0.18901,-22.47999,9.02372,0.0,1.0,0.0,0.0,0.0,0.0', 'run_id': 'run_0000', 'target_id': '587722984435351614'}
